In this Notebook I will only analyse the Cambridgeshire stop-and-search dataset (the visualisations with stop-an-search operations and arrests from searching for drugs is in the project diary)

## Importing the data

In [1]:
# importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.rcParams['figure.figsize'] = (15, 15) # Reset the base size of figures so they're large enough to be useful.
mpl.rcParams['figure.figsize'] = (25, 25) # Reset the base size of figures so they're large enough to be useful.
mpl.rcParams.update({'font.size': 24}) # Reset fontsize to a legible one

import scipy.stats

import numpy as np

#import re

import folium
#import glob
import os

In [2]:
!ls -l ../EMA_data/stop-and-search/2017-10

total 3240
-rwxrwxrwx 1 oustudent users 1473234 Mar 21 21:34 2017-10-cambridgeshire-outcomes.csv
-rwxrwxrwx 1 oustudent users   19219 Mar 21 21:34 2017-10-cambridgeshire-stop-and-search.csv
-rwxrwxrwx 1 oustudent users 1821617 Mar 21 21:34 2017-10-cambridgeshire-street.csv


In [3]:
path = '../EMA_data/stop-and-search'

file_list=[]

for subdir, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('stop-and-search.csv'):
            print(os.path.join(subdir, file))

../EMA_data/stop-and-search/2017-06/2017-06-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2017-07/2017-07-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2017-08/2017-08-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2017-09/2017-09-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2017-10/2017-10-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2017-11/2017-11-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2017-12/2017-12-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2018-01/2018-01-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2018-02/2018-02-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2018-03/2018-03-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2018-04/2018-04-cambridgeshire-stop-and-search.csv
../EMA_data/stop-and-search/2018-05/2018-05-cambridgeshire-stop-and-search.csv


In [4]:
# the whole year Cambridgeshire Stop and Search
path = '../EMA_data/stop-and-search'

file_list=[]

for subdir, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('stop-and-search.csv'):
            sas_df = pd.read_csv(os.path.join(subdir, file), index_col=None, header = 0)
            file_list.append(sas_df)
        
sas_df = pd.concat(file_list, axis=0, ignore_index=True)
# sas_df[['Gender']] = sas_df[['Gender']].fillna('Unknown')
sas_df.head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing
0,Person search,2017-06-01T00:53:37+00:00,NaN,NaN,52.390692,0.211098,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False
1,Person and Vehicle search,2017-06-01T00:57:39+00:00,NaN,NaN,52.390692,0.211098,Male,10-17,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False
2,Person and Vehicle search,2017-06-01T12:13:01+00:00,NaN,NaN,52.204686,0.136415,Male,25-34,Black/African/Caribbean/Black British - Caribbean,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,True
3,Person and Vehicle search,2017-06-02T01:22:11+00:00,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False
4,Person search,2017-06-02T01:46:30+00:00,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False


In [5]:
# how many NaN values in Longitude (and Latitude) columns

sas_df.isna().sum()
# only five missing
# Gender missing 54 times

Type                                           0
Date                                           0
Part of a policing operation                1002
Policing operation                          1037
Latitude                                       5
Longitude                                      5
Gender                                        54
Age range                                     70
Self-defined ethnicity                       115
Officer-defined ethnicity                     96
Legislation                                    5
Object of search                               0
Outcome                                        0
Outcome linked to object of search           662
Removal of more than just outer clothing      59
dtype: int64

In [6]:
# Object of search (never is a missing value)
sas_df['Object of search'].unique()

array(['Controlled drugs', 'Offensive weapons',
       'Article for use in theft', 'Articles for use in criminal damage',
       'Evidence of offences under the Act', 'Stolen goods', 'Firearms'],
      dtype=object)

In [7]:
# Convert Date to datetime
sas_df['Date'] = pd.to_datetime(sas_df['Date'])
sas_df.head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing
0,Person search,2017-06-01 00:53:37,NaN,NaN,52.390692,0.211098,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False
1,Person and Vehicle search,2017-06-01 00:57:39,NaN,NaN,52.390692,0.211098,Male,10-17,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False
2,Person and Vehicle search,2017-06-01 12:13:01,NaN,NaN,52.204686,0.136415,Male,25-34,Black/African/Caribbean/Black British - Caribbean,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,True
3,Person and Vehicle search,2017-06-02 01:22:11,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False
4,Person search,2017-06-02 01:46:30,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False


In [8]:
#sas_df=sas_df.rename(columns = {'Date':'Month'})
#sas_df.head()

In [9]:
sas_df['Month'] = sas_df['Date'].dt.normalize()
sas_df.head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,Month
0,Person search,2017-06-01 00:53:37,NaN,NaN,52.390692,0.211098,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False,2017-06-01
1,Person and Vehicle search,2017-06-01 00:57:39,NaN,NaN,52.390692,0.211098,Male,10-17,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False,2017-06-01
2,Person and Vehicle search,2017-06-01 12:13:01,NaN,NaN,52.204686,0.136415,Male,25-34,Black/African/Caribbean/Black British - Caribbean,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,True,2017-06-01
3,Person and Vehicle search,2017-06-02 01:22:11,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False,2017-06-02
4,Person search,2017-06-02 01:46:30,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False,2017-06-02


In [10]:
sas_df['Month'] = sas_df['Month'].dt.to_period('M')
sas_df.head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,Month
0,Person search,2017-06-01 00:53:37,NaN,NaN,52.390692,0.211098,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False,2017-06
1,Person and Vehicle search,2017-06-01 00:57:39,NaN,NaN,52.390692,0.211098,Male,10-17,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False,2017-06
2,Person and Vehicle search,2017-06-01 12:13:01,NaN,NaN,52.204686,0.136415,Male,25-34,Black/African/Caribbean/Black British - Caribbean,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,True,2017-06
3,Person and Vehicle search,2017-06-02 01:22:11,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False,2017-06
4,Person search,2017-06-02 01:46:30,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False,2017-06


In [11]:
sas_df.dtypes

Type                                                object
Date                                        datetime64[ns]
Part of a policing operation                        object
Policing operation                                 float64
Latitude                                           float64
Longitude                                          float64
Gender                                              object
Age range                                           object
Self-defined ethnicity                              object
Officer-defined ethnicity                           object
Legislation                                         object
Object of search                                    object
Outcome                                             object
Outcome linked to object of search                  object
Removal of more than just outer clothing            object
Month                                               object
dtype: object

In [12]:
# rows
print(sas_df.shape[0])
# columns
print(sas_df.shape[1])

1037
16


In [13]:
# Check for later _ stop and search June 2017
jun_df = pd.read_csv("../EMA_data/stop-and-search/2017-06/2017-06-cambridgeshire-stop-and-search.csv")

jun_df.head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing
0,Person search,2017-06-01T00:53:37+00:00,NaN,NaN,52.390692,0.211098,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False
1,Person and Vehicle search,2017-06-01T00:57:39+00:00,NaN,NaN,52.390692,0.211098,Male,10-17,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False
2,Person and Vehicle search,2017-06-01T12:13:01+00:00,NaN,NaN,52.204686,0.136415,Male,25-34,Black/African/Caribbean/Black British - Caribbean,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,True
3,Person and Vehicle search,2017-06-02T01:22:11+00:00,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False
4,Person search,2017-06-02T01:46:30+00:00,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False


In [14]:
jun_df['Object of search'].value_counts()

Controlled drugs                       42
Article for use in theft               10
Offensive weapons                       7
Articles for use in criminal damage     4
Stolen goods                            3
Evidence of offences under the Act      1
Name: Object of search, dtype: int64

In [15]:
sas_df['Object of search'].value_counts()

Controlled drugs                       599
Article for use in theft               199
Offensive weapons                      165
Stolen goods                            51
Articles for use in criminal damage     11
Evidence of offences under the Act       7
Firearms                                 5
Name: Object of search, dtype: int64

In [16]:
# Check for later _ stop and search Jul 2017
jul_df = pd.read_csv("../EMA_data/stop-and-search/2017-07/2017-07-cambridgeshire-stop-and-search.csv")

jul_df.head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing
0,Person search,2017-07-01T01:11:24+00:00,NaN,NaN,52.574809,-0.241137,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False
1,Person search,2017-07-01T07:43:43+00:00,NaN,NaN,52.667461,0.159571,Male,25-34,White - Any other White background,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,True
2,Person search,2017-07-01T19:58:22+00:00,NaN,NaN,52.570945,-0.243696,Female,25-34,White - Any other White background,White,Police and Criminal Evidence Act 1984 (section 1),Article for use in theft,Local resolution,NaN,False
3,Person search,2017-07-01T23:23:09+00:00,NaN,NaN,52.266023,0.562663,Male,18-24,Other ethnic group - Not stated,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Nothing found - no further action,NaN,False
4,Vehicle search,2017-07-02T14:45:51+00:00,NaN,NaN,52.211446,0.143443,NaN,NaN,NaN,NaN,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Suspect arrested,True,NaN


In [17]:
jul_df['Object of search'].value_counts()

Controlled drugs                      51
Offensive weapons                     13
Article for use in theft               9
Evidence of offences under the Act     1
Name: Object of search, dtype: int64

In [18]:
# how many searches for each object of search type by month
# first problem - counts searches for each search each day
# second problem - counts searches for each day in a month
# should be fine when counts searches for each month
sas_df.groupby(['Month', 'Object of search']).size()
search_types_df = sas_df.groupby(['Month', 'Object of search']).size().to_frame('Total searches per month').reset_index().sort_values(['Month', 'Total searches per month'], ascending=[True, False])
search_types_df.head()

,Month,Object of search,Total searches per month
2,2017-06,Controlled drugs,42
0,2017-06,Article for use in theft,10
4,2017-06,Offensive weapons,7
1,2017-06,Articles for use in criminal damage,4
5,2017-06,Stolen goods,3


In [19]:
monthly_searches_df = search_types_df.pivot('Month','Object of search', 'Total searches per month')
#monthly_searches_df.index = pd.to_datetime(monthly_searches_df.index)
monthly_searches_df.fillna(0, inplace=True)
#monthly_searches_df.astype(int)
monthly_searches_df


Object of search,Article for use in theft,Articles for use in criminal damage,Controlled drugs,Evidence of offences under the Act,Firearms,Offensive weapons,Stolen goods
Month,,,,,,,
2017-06,10.0,4.0,42.0,1.0,0.0,7.0,3.0
2017-07,9.0,0.0,51.0,1.0,0.0,13.0,0.0
2017-08,25.0,0.0,58.0,0.0,0.0,6.0,7.0
2017-09,12.0,0.0,44.0,5.0,2.0,11.0,8.0
2017-10,20.0,1.0,42.0,0.0,0.0,15.0,6.0
2017-11,13.0,0.0,39.0,0.0,0.0,14.0,2.0
2017-12,17.0,0.0,49.0,0.0,0.0,23.0,2.0
2018-01,11.0,0.0,52.0,0.0,1.0,15.0,6.0
2018-02,10.0,0.0,50.0,0.0,0.0,17.0,6.0


In [20]:
#chi2, p, _, _ = scipy.stats.chi2_contingency(monthly_searches_df)
#chi2, p

In [21]:
sas_df['Outcome'].unique()

array(['Nothing found - no further action',
       'Offender given drugs possession warning',
       'Article found - Detailed outcome unavailable',
       'Suspect summonsed to court', 'Suspect arrested',
       'Local resolution', 'Offender cautioned',
       'Offender given penalty notice', 'A no further action disposal',
       'Arrest', 'Summons / charged by post', 'Community resolution',
       'Khat or Cannabis warning', 'Caution (simple or conditional)'],
      dtype=object)

In [22]:
all_outcomes = sas_df['Outcome'].unique().tolist()

In [23]:
arrests = ['Arrest', 'Suspect arrested']
drug_arrests_df = sas_df.loc[(sas_df['Object of search'] == 'Controlled drugs') 
                             & ((sas_df['Outcome'] == 'Arrest') | (sas_df['Outcome'] == 'Suspect arrested'))]



drug_arrests_df['Outcome'].count()
# 105 arrests for the year (for controlled drugs)

105

In [24]:
drug_arrests_df['Outcome linked to object of search'].value_counts(dropna=False)

True     60
False    32
NaN      13
Name: Outcome linked to object of search, dtype: int64

Check how many arrests were linked to the object of search (which is controlled drugs)

In [25]:
arrests_df = pd.DataFrame(drug_arrests_df['Outcome linked to object of search'].value_counts(dropna=False))
arrests_df = arrests_df.T
arrests_df = arrests_df.rename_axis(None)
arrests_df.columns = ['Arrest linked to object of search', 'Arrest NOT linked to object of search', 'Unknown']
arrests_df.set_index(['Arrest linked to object of search', 'Arrest NOT linked to object of search', 'Unknown'], inplace=True)
arrests_df

,,
Arrest linked to object of search,Arrest NOT linked to object of search,Unknown
60,32,13


In [26]:
# positive outcomes - where there is some kind of action taken against the suspect (person searched)
sas_df['Object of search'].value_counts()

Controlled drugs                       599
Article for use in theft               199
Offensive weapons                      165
Stolen goods                            51
Articles for use in criminal damage     11
Evidence of offences under the Act       7
Firearms                                 5
Name: Object of search, dtype: int64

In [27]:
#pos_drug_outcomes_df.head()

In [28]:
sas_df['Outcome'].unique()

array(['Nothing found - no further action',
       'Offender given drugs possession warning',
       'Article found - Detailed outcome unavailable',
       'Suspect summonsed to court', 'Suspect arrested',
       'Local resolution', 'Offender cautioned',
       'Offender given penalty notice', 'A no further action disposal',
       'Arrest', 'Summons / charged by post', 'Community resolution',
       'Khat or Cannabis warning', 'Caution (simple or conditional)'],
      dtype=object)

<p><a href="https://www.police.uk/about-this-site/stop-and-search/">From April 2018</a>:</p>

<p>&nbsp;</p>

<table class="responsive">
	<thead>
		<tr>
			<th>New Category</th>
			<th>Old Category</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td>Arrest (as is currently required)</td>
			<td>Suspect Arrested</td>
		</tr>
		<tr>
			<td>Summons / Charged by post</td>
			<td>Suspect summons to court</td>
		</tr>
		<tr>
			<td>Caution (simple or conditional)</td>
			<td>Offender Cautioned</td>
		</tr>
		<tr>
			<td>Khat or cannabis warning</td>
			<td>Offender given drugs possession warning</td>
		</tr>
		<tr>
			<td>Penalty notice for disorder</td>
			<td>Offender given penalty notice</td>
		</tr>
		<tr>
			<td>Community resolution</td>
			<td>Local resolution</td>
		</tr>
		<tr>
			<td>A no further action disposal</td>
			<td>Article found &ndash; detailed outcome unavailable*</td>
		</tr>
	</tbody>
</table>

<p>*Unfortunately this category does not map perfectly, as such they are not used as equivalencies.</p>

<p>&nbsp;</p>


In [29]:
pos_outcomes = ['Arrest', 'Suspected Arrested', 'Summons / charged by post', 'Suspect summons to court', 
                'Caution (simple or conditional)', 'Offender Cautioned', 'Khat or cannabis warning',
                'Offender given drugs possession warning', 'Penalty notice for disorder', 'Offender given penalty notice',
                'Community resolution', 'Local resolution', 'Article found – detailed outcome unavailable*']

neg_outcomes = ['Nothing found - no further action', 'A no further action disposal']

In [30]:
pos_drug_outcomes_df = sas_df.loc[~(sas_df['Outcome'].isin(neg_outcomes)) & (sas_df['Object of search'] == 'Controlled drugs')]

#sas_df.loc[(sas_df['Object of search'] == 'Controlled drugs') 
#                             & ((sas_df['Outcome'] == 'Arrest') | (sas_df['Outcome'] == 'Suspect arrested'))]
pos_drug_outcomes_df['Outcome'].unique()

array(['Offender given drugs possession warning',
       'Article found - Detailed outcome unavailable',
       'Suspect summonsed to court', 'Suspect arrested',
       'Offender cautioned', 'Offender given penalty notice',
       'Local resolution', 'Summons / charged by post',
       'Community resolution', 'Arrest', 'Khat or Cannabis warning',
       'Caution (simple or conditional)'], dtype=object)

In [31]:
pos_drug_outcomes_df.shape

(292, 16)

In [32]:
sas_df['Outcome'].isin(pos_outcomes).sum()


overall_pos_drug_outcomes_df = sas_df.loc[~(sas_df['Outcome'].isin(neg_outcomes))]

#sas_df.loc[(sas_df['Object of search'] == 'Controlled drugs') 
#                             & ((sas_df['Outcome'] == 'Arrest') | (sas_df['Outcome'] == 'Suspect arrested'))]
overall_pos_drug_outcomes_df.shape

(445, 16)

In [33]:
neg_outcomes = ['Nothing found - no further action', 'A no further action disposal']
# sas_df.loc[~sas_df['Outcome'].isin(neg_outcomes)]
sas_df.loc[~(sas_df['Outcome'].isin(neg_outcomes)) & (sas_df['Object of search'] == 'Controlled drugs')].head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,Month
3,Person and Vehicle search,2017-06-02 01:22:11,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False,2017-06
4,Person search,2017-06-02 01:46:30,NaN,NaN,52.311854,0.390497,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False,2017-06
8,Person search,2017-06-03 13:15:46,NaN,NaN,52.208385,0.129772,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Article found - Detailed outcome unavailable,True,False,2017-06
9,Person search,2017-06-03 18:00:00,NaN,NaN,52.208385,0.129772,Male,10-17,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Suspect summonsed to court,NaN,NaN,2017-06
18,Person search,2017-06-06 19:01:16,NaN,NaN,52.620136,-0.268866,Male,10-17,Other ethnic group - Not stated,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,True,False,2017-06


In [34]:
sas_df.count()

Type                                        1037
Date                                        1037
Part of a policing operation                  35
Policing operation                             0
Latitude                                    1032
Longitude                                   1032
Gender                                       983
Age range                                    967
Self-defined ethnicity                       922
Officer-defined ethnicity                    941
Legislation                                 1032
Object of search                            1037
Outcome                                     1037
Outcome linked to object of search           375
Removal of more than just outer clothing     978
Month                                       1037
dtype: int64

<p><strong>Overall</strong></p>

<p>Overall 1037 searches from which positive outcomes - 445, negative -592. 42.9 % positive outcomes</p>

<p>Arrests made 181 times</p>

<p>&nbsp;</p>

<p>&nbsp;</p>

<p><strong>Object of search&nbsp; - controlled drugs</strong></p>

<p>When looking at controlled drugs - overall 599 searches -&nbsp; positive outcomes - 292. 48.7 %.</p>

<p>Arrests made 105 times</p>

<p>&nbsp;</p>

<p>&nbsp;</p>


In [35]:
(181/1037)*100
(105/599)*100

17.529215358931552

In [36]:
# Overall arrests
arrestList = ['Arrest', 'Suspect arrested']
overall_arrests_df =  sas_df.loc[sas_df['Outcome'].isin(arrestList)]
overall_arrests_df.head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,Month
10,Person search,2017-06-03 19:30:00,NaN,NaN,52.208385,0.129772,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,Suspect arrested,False,NaN,2017-06
12,Person and Vehicle search,2017-06-04 15:03:39,NaN,NaN,52.173351,0.112317,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Police and Criminal Evidence Act 1984 (section 1),Article for use in theft,Suspect arrested,False,False,2017-06
15,Person search,2017-06-04 19:09:29,NaN,NaN,52.547130,-0.281194,Male,over 34,Other ethnic group - Not stated,White,Police and Criminal Evidence Act 1984 (section 1),Article for use in theft,Suspect arrested,False,False,2017-06
20,Vehicle search,2017-06-07 09:54:35,NaN,NaN,52.663840,0.165796,NaN,NaN,NaN,NaN,Police and Criminal Evidence Act 1984 (section 1),Articles for use in criminal damage,Suspect arrested,NaN,NaN,2017-06
22,Person search,2017-06-07 18:58:44,NaN,NaN,52.199025,0.147075,Male,10-17,Black/African/Caribbean/Black British - African,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Suspect arrested,True,True,2017-06


In [37]:
overall_arrests_df['Outcome linked to object of search'].value_counts(dropna=False)

False    80
True     64
NaN      37
Name: Outcome linked to object of search, dtype: int64